In [15]:
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import nltk

np.random.seed(2018)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
raw_Data = pd.read_csv('million-headlines/abcnews-date-text.csv', error_bad_lines=False)
data = raw_Data[['headline_text']]
data['index'] = data.index
documents = data
documents.head(5)

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [21]:
def lemmatize_stemming(text):
    return SnowballStemmer("english").stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [22]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [24]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [25]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [26]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [27]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4015, 1)]

In [28]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


In [29]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


In [30]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [31]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.016*"deal" + 0.014*"driver" + 0.013*"flood" + 0.012*"street" + 0.010*"babi" + 0.009*"east" + 0.009*"leav"
Topic: 1 
Words: 0.025*"world" + 0.019*"test" + 0.016*"interview" + 0.015*"australia" + 0.012*"leagu" + 0.010*"close" + 0.010*"darwin" + 0.010*"game" + 0.009*"take" + 0.009*"star"
Topic: 2 
Words: 0.018*"council" + 0.018*"countri" + 0.016*"nation" + 0.015*"tasmanian" + 0.015*"hour" + 0.014*"state" + 0.012*"plan" + 0.011*"citi" + 0.011*"say" + 0.011*"news"
Topic: 3 
Words: 0.023*"perth" + 0.020*"women" + 0.018*"sydney" + 0.018*"miss" + 0.013*"protest" + 0.012*"guilti" + 0.011*"search" + 0.011*"water" + 0.011*"john" + 0.010*"continu"
Topic: 4 
Words: 0.028*"elect" + 0.019*"say" + 0.017*"live" + 0.017*"famili" + 0.016*"chang" + 0.011*"park" + 0.011*"polit" + 0.010*"vote" + 0.010*"leader" + 0.010*"christma"
Topic: 5 
Words: 0.035*"trump" + 0.028*"queensland" + 0.015*"tasmania" + 0.014*"turnbul" + 0.013*"say" + 0.012*"s

In [33]:
#tfidf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.018*"polic" + 0.016*"charg" + 0.015*"murder" + 0.011*"woman" + 0.010*"court" + 0.009*"shoot" + 0.009*"jail" + 0.008*"arrest" + 0.008*"search" + 0.008*"guilti"
Topic: 1 Word: 0.014*"rural" + 0.014*"market" + 0.012*"news" + 0.009*"drum" + 0.009*"west" + 0.008*"weather" + 0.008*"share" + 0.007*"busi" + 0.007*"coast" + 0.007*"farmer"
Topic: 2 Word: 0.009*"australia" + 0.009*"queensland" + 0.008*"elect" + 0.006*"christma" + 0.006*"celebr" + 0.006*"northern" + 0.006*"novemb" + 0.005*"abbott" + 0.005*"malcolm" + 0.005*"stori"
Topic: 3 Word: 0.007*"world" + 0.006*"final" + 0.006*"win" + 0.006*"octob" + 0.006*"tuesday" + 0.006*"street" + 0.006*"open" + 0.005*"australia" + 0.005*"outback" + 0.005*"australian"
Topic: 4 Word: 0.008*"kill" + 0.007*"john" + 0.007*"michael" + 0.006*"crash" + 0.005*"decemb" + 0.005*"attack" + 0.005*"dead" + 0.005*"plane" + 0.005*"bomb" + 0.005*"syria"
Topic: 5 Word: 0.007*"mental" + 0.007*"sexual" + 0.006*"august" + 0.006*"leagu" + 0.005*"rugbi" + 0.0

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))